**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Start

Deployment and Service

https://kubernetes.io/docs/concepts/services-networking/connect-applications-service/

In [15]:
kubectl get pods -A

NAMESPACE     NAME                               READY   STATUS    RESTARTS   AGE
default       my-nginx-5b56ccd65f-8krqn          1/1     Running   0          45m
default       my-nginx-5b56ccd65f-s8qwn          1/1     Running   0          45m
kube-system   coredns-74ff55c5b-bx9lj            1/1     Running   0          127m
kube-system   etcd-minikube                      1/1     Running   0          127m
kube-system   kube-apiserver-minikube            1/1     Running   0          127m
kube-system   kube-controller-manager-minikube   1/1     Running   0          127m
kube-system   kube-proxy-2rf67                   1/1     Running   0          127m
kube-system   kube-scheduler-minikube            1/1     Running   0          127m
kube-system   storage-provisioner                1/1     Running   0          128m


# Warning

**we install software from an unknown source**

In [2]:
kubectl create -f https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/service/networking/run-my-nginx.yaml

deployment.apps/my-nginx created


In [3]:
kubectl create -f https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/service/networking/nginx-svc.yaml

service/my-nginx created


In [6]:
kubectl get all

NAME                            READY   STATUS    RESTARTS   AGE
pod/my-nginx-5b56ccd65f-8krqn   1/1     Running   0          21s
pod/my-nginx-5b56ccd65f-s8qwn   1/1     Running   0          21s

NAME                 TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1     <none>        443/TCP   82m
service/my-nginx     ClusterIP   10.97.45.87   <none>        80/TCP    19s

NAME                       READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/my-nginx   2/2     2            2           21s

NAME                                  DESIRED   CURRENT   READY   AGE
replicaset.apps/my-nginx-5b56ccd65f   2         2         2       21s


In [7]:
curl https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/service/networking/nginx-svc.yaml

apiVersion: v1
kind: Service
metadata:
  name: my-nginx
  labels:
    run: my-nginx
spec:
  ports:
  - port: 80
    protocol: TCP
  selector:
    run: my-nginx


In [8]:
curl https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/service/networking/run-my-nginx.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: my-nginx
spec:
  selector:
    matchLabels:
      run: my-nginx
  replicas: 2
  template:
    metadata:
      labels:
        run: my-nginx
    spec:
      containers:
      - name: my-nginx
        image: nginx
        ports:
        - containerPort: 80



In [9]:
kubectl get service my-nginx

NAME       TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)   AGE
my-nginx   ClusterIP   10.97.45.87   <none>        80/TCP    3m30s


In [10]:
kubectl get service my-nginx -o yaml

apiVersion: v1
kind: Service
metadata:
  creationTimestamp: "2021-01-26T09:39:08Z"
  labels:
    run: my-nginx
  managedFields:
  - apiVersion: v1
    fieldsType: FieldsV1
    fieldsV1:
      f:metadata:
        f:labels:
          .: {}
          f:run: {}
      f:spec:
        f:ports:
          .: {}
          k:{"port":80,"protocol":"TCP"}:
            .: {}
            f:port: {}
            f:protocol: {}
            f:targetPort: {}
        f:selector:
          .: {}
          f:run: {}
        f:sessionAffinity: {}
        f:type: {}
    manager: kubectl-create
    operation: Update
    time: "2021-01-26T09:39:08Z"
  name: my-nginx
  namespace: default
  resourceVersion: "3924"
  uid: 94190287-0839-4242-84fe-8b29c0db4fa1
spec:
  clusterIP: 10.97.45.87
  clusterIPs:
  - 10.97.45.87
  ports:
  - port: 80
    protocol: TCP
    targetPort: 80
  selector:
    run: my-nginx
  sessionAffinity: None
  type: ClusterIP
status:
  loadBalancer: {}


In [ ]:
kubectl get service my-nginx -o jsonpath='{.spec.clusterIP}'

In [13]:
IP=$(kubectl get service my-nginx -o jsonpath='{.spec.clusterIP}')
curl $IP

<!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
    body {
        width: 35em;
        margin: 0 auto;
        font-family: Tahoma, Verdana, Arial, sans-serif;
    }
</style>
</head>
<body>
<h1>Welcome to nginx!</h1>
<p>If you see this page, the nginx web server is successfully installed and
working. Further configuration is required.</p>

<p>For online documentation and support please refer to
<a href="http://nginx.org/">nginx.org</a>.<br/>
Commercial support is available at
<a href="http://nginx.com/">nginx.com</a>.</p>

<p><em>Thank you for using nginx.</em></p>
</body>
</html>


In [14]:
echo http://$IP

http://10.97.45.87
